In [3]:
import os

# Ubicación del archivo CURATE_data
dat_file = [file for file in os.listdir('.') if file.endswith('.dat') and file.startswith('CURATE_data')][0]

# Leer las 4 primeras líneas del archivo DAT para obtener cita y command line
with open(dat_file, 'r', errors="replace") as datfile:
    first_line = datfile.readline()
    second_line = datfile.readline()
    third_line = datfile.readline()
    comand_line = datfile.readline()
    dat_content = datfile.read()
    start_pos = dat_content.find("Starting")-3
    end_pos = dat_content.find("Time CURATE:", start_pos)
    end_pos = dat_content.find("\n", end_pos) # busco el siguiente salto de línea
    first_two_lines = first_line + second_line +'\n' + comand_line
    curated_data = dat_content[start_pos:end_pos]

def insert_newlines(string, every=100):
    """Inserts a newline character every `every` characters in the string."""
    return '\n'.join(string[i:i+every] for i in range(0, len(string), every)) 

# Insertar saltos de línea para que las dos primeras líneas tengan una anchura máxima
first_two_lines = insert_newlines(first_two_lines, 126)  

# Insertar saltos de línea para que cada línea tenga una anchura máxima 
lines = curated_data.split('\n')
new_lines = []
for line in lines:
    if line.startswith('- '):
        # Reemplazar los primeros 4 espacios por 8 espacios para mantener la sangría original
        new_line = ' ' * 8 + line[4:]
    else:
        new_line = line
    new_lines.append(insert_newlines(new_line))
curated_data = '\n'.join(new_lines)

# Ubicación del archivo GENERATE
generate_dat_file = [file for file in os.listdir('.') if file.endswith('.dat') and file.startswith('GENERATE_data')][0]

# Leer la sección deseada del archivo DAT y guardarla en una variable
with open(generate_dat_file, 'r', errors="replace") as datfile:
    dat_content = datfile.read()
    start_pos = dat_content.find("Starting")-3
    end_pos = dat_content.find("Heatmap generation:")
    end_pos = dat_content.find("\n", end_pos) + 1 # incluir la línea "Heatmap generation:" en la sección
    generated_data = dat_content[start_pos:end_pos]

# Insertar saltos de línea para que cada línea tenga una anchura máxima de 80 caracteres
lines = generated_data.split('\n')
new_lines = []
for line in lines:
    if line.startswith('- '):
        # Agregar 4 espacios al principio de la línea que empieza con "- "
        new_line = '    ' + line
    else:
        new_line = line
    new_lines.append(insert_newlines(new_line))
generated_data = '\n'.join(new_lines)

# Ubicación del archivo VERIFY_data
verify_file = [file for file in os.listdir('.') if file.endswith('.dat') and file.startswith('VERIFY_data')][0]

# Leer el contenido completo del archivo VERIFY_data
with open(verify_file, 'r', errors="replace") as verifyfile:
    verify_content = verifyfile.read()
    start_pos = verify_content.find("Starting tests")-3
    end_pos = verify_content.find("Time VERIFY:", start_pos)
    end_pos = verify_content.find("\n", end_pos) # busco el siguiente salto de línea
    verify_data = verify_content[start_pos:end_pos]

lines = verify_data.split('\n')
new_lines = []
for line in lines:
    if line.startswith('- '):
        # Agregar 4 espacios al principio de la línea que empieza con "- "
        new_line = '    ' + line
    else:
        new_line = line
    new_lines.append(insert_newlines(new_line, 100))
verify_data = '\n'.join(new_lines)

# Ubicación del archivo PREDICT_data
predict_file = [file for file in os.listdir('.') if file.endswith('.dat') and file.startswith('PREDICT_data')][0]

# Leer el contenido completo del archivo PREDICT_data
with open(predict_file, 'r', errors="replace") as predictfile:
    predict_content = predictfile.read()
    start_pos = predict_content.find("Representation")-3
    end_pos = predict_content.find("Time PREDICT:", start_pos)
    end_pos = predict_content.find("\n", end_pos) # busco el siguiente salto de línea
    predict_data = predict_content[start_pos:end_pos]
    
lines = predict_data.split('\n')
new_lines = []
for line in lines:
    if line.startswith('- '):
        # Agregar 4 espacios al principio de la línea que empieza con "- "
        new_line = '    ' + line
    else:
        new_line = line
    new_lines.append(insert_newlines(new_line))
predict_data = '\n'.join(new_lines)

# Buscar imagenes
predict_dir = './PREDICT'
dir_depth = 2  # Buscar hasta dos niveles de subcarpetas
matching_files = []

for root, dirs, files in os.walk(predict_dir):
    depth = root[len(predict_dir) + len(os.path.sep):].count(os.path.sep)
    if depth <= dir_depth:
        for file in sorted(files, reverse=True):
            if 'PFI' in file and file.endswith('.png'):
                matching_files.append(os.path.join(root, file))

if matching_files:
    # Si hay archivos que coinciden con el patrón, genere el código HTML para cada imagen
    html_predict_png = ''.join(f'<img src="{image_path}" alt="" class="img-predict"/>' for image_path in matching_files)

# Generar los HTML
first_two_lines_html = '<div class="dat-content"><pre>' + first_two_lines + '</pre></div>'   
curate_html = '<div class="dat-content"><pre>' + curated_data + '</pre></div>'
predict_html = '<div class="predict-content"><pre>' + predict_data + '</pre></div>'
verify_html = '<div class="verify-content"><pre>' + verify_data + '</pre></div>'
generate_dat_html = '<div class="dat-content"><pre>' + generated_data + '</pre></div>'

# Generar el HTML del REPORT completo
report_html = """
<h1 style="text-align: left; margin-bottom: 0.5em;">
    Report
    <img src="Robert_logo.jpg" alt="" style="float:right; width:25%;" />
    <br>
    <hr class="black">
    <span style="font-weight:bold;"></span>
</h1>
<p style="text-align: center;"></p>
<p style="text-align: justify;"
{}
<hr>
<p><span style="font-weight:bold;">
    <img src="content-curation.png" alt="" style="width:20px; height:20px; margin-right:5px;">
    CURATE 
</span></p>
{}
<img src="Pearson_heatmap.png" alt="" />
<p>
<hr>
<p><span style="font-weight:bold;">
    <img src="cpu.png" alt="" style="width:20px; height:20px; margin-right:5px;">
    GENERATE 
</span></p>
{}
<img src="NO_PFI.png" alt="" />
<img src="PFI.png" alt="" />
<p>
<hr>
<p><span style="font-weight:bold;">
    <img src="analysis.png" alt="" style="width:20px; height:20px; margin-right:5px;">
    PREDICT 
</span></p>
{}
{}
<p>
<hr>
<p><span style="font-weight:bold;">
    <img src="verified.png" alt="" style="width:20px; height:20px; margin-right:5px;">
    VERIFY 
</span></p>
{}
<img src="VERIFY_tests_No_PFI.png" alt="" />
<img src="VERIFY_tests_PFI.png" alt="" />
<p>
<hr>
<p style="text-align: justify;"><span style="font-weight:bold;">AQME-ROBERT</span></p>
<img src="AQME_ROBERT.png" alt="" />
</span>
""".format(first_two_lines_html, curate_html,generate_dat_html,predict_html,html_predict_png,verify_html)

# Guardar el HTML generado en un archivo
with open("report.html", "w") as htmlfile:
    htmlfile.write(report_html)
    
# Generar archivo CSS
css_content = """
body {
  font-size: 12px;
}
@page :first {
    @top-left {
        content: "";
    }
    @top-right {
        content: "";
    }
    @bottom-left {
        content: "ROBERT v 0.0.1";
        font-size: 8pt;
        position: fixed;
        left: 0;
        bottom: 0;
    }
    @bottom-right {
        content: counter(page) " of " counter(pages);
        font-size: 8pt;
        position: fixed;
        right: 0;
        bottom: 0;
    }
}
@page {
    size: A4;
    margin: 2cm;
    @top-left {
        content: "ROBERT report";
        font-size: 8pt;
        position: fixed;
        left: 0;
        top: 0;
    }
    @top-right {
        content: "CSV file name";
        font-size: 8pt;
        position: fixed;
        left: 0;
        top: 0;
    }
    @bottom-left {
        content: "ROBERT v 0.0.1";
        font-size: 8pt;
        position: fixed;
        left: 0;
        bottom: 0;
    }
    @bottom-right {
        content: counter(page) " of " counter(pages);
        font-size: 8pt;
        position: fixed;
        right: 0;
        bottom: 0;
    }
}
* {
    font-family: Arial, sans-serif;
}
.dat-content {
    width: 50%;
    max-width: 595pt;
    overflow-x: auto;
    line-height: 1.2;
}

img[src="Robert_logo.jpg"] {
	float: right;
	width: 50%;
}
img[src*="Pearson"] {
	display: inline-block;
	vertical-align: bottom;
	max-width: 48%;
	margin-left: 10px;
	margin-bottom: -5px;
}

img[src*="PFI"] {
	display: inline-block;
	vertical-align: bottom;
	max-width: 48%;
	margin-left: 10px;
	margin-bottom: -5px;
}

img[src*="PFI"]:first-child {
	margin-right: 10%;
}
.img-predict {
    margin-top: 20px;
}
hr.black {
  border: none;
  height: 1px;
  background-color: black;
}

hr {
  border: none;
  height: 1px;
  background-color: gray;
}

body:before {
  top: 1.2cm;
}
"""
# Escribir el contenido del archivo CSS en un archivo
with open("report.css", "w") as cssfile:
    cssfile.write(css_content)


In [4]:
from pathlib import Path
from weasyprint import HTML


def makepdf(html, css_files=None):
    """Generate a PDF file from a string of HTML."""
    htmldoc = HTML(string=html, base_url="")
    if css_files:
        htmldoc = htmldoc.write_pdf(stylesheets=css_files)
    else:
        htmldoc = htmldoc.write_pdf()
    return htmldoc


def run():
    """Command runner."""
    html_file = "report.html"  # Ruta del archivo HTML
    css_files = ["report.css"]  # Ruta del archivo CSS
    outfile = "Report.pdf"  # Ruta del archivo PDF de salida
    html = Path(html_file).read_text()
    pdf = makepdf(html, css_files=css_files)
    Path(outfile).write_bytes(pdf)


if __name__ == "__main__":
    run()
